## **INSTALL MODULES (FOR MODEL AND DATASETS)**

In [ ]:
# #For Google Collab
# import os, sys
# from google.colab import drive
# drive.mount('/content/drive')
# nb_path = '/content/notebooks'
# os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
# sys.path.insert(0, nb_path)

In [ ]:
# !pip install --target=$nb_path git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

In [ ]:
# !pip install --target=$nb_path datasets

In [ ]:
# pip install git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

In [ ]:
# pip install datasets

## **Download dataset (streaming mode) for test**

In [ ]:
# FOR EXECUTING ALL THE FILES AND GENERATING MANY CSVs with filtered rows
'''

LOAD MODEL (function)
First execute a for loop with all the shards/subsets (function)
For each one, LOAD DATA and execute the pipeline
Generate csv
decide 


'''

In [48]:
from datasets import load_dataset

#all folders in espnet/yodas (224 subsets)
# configs = ['aa000', 'ab000', 'af000', 'ak000', 'am000', 'ar000', 'as000', 'ay000', 'az000', 'ba000', 'be000', 'bg000', 'bh000', 'bi000', 'bm000', 'bn000', 'bo000', 'br000', 'bs000', 'ca000', 'co000', 'cr000', 'cs000', 'cy000', 'da000', 'de000', 'de100', 'de101', 'de102', 'dz000', 'ee000', 'el000', 'en000', 'en001', 'en002', 'en003', 'en004', 'en005', 'en100', 'en101', 'en102', 'en103', 'en104', 'en105', 'en106', 'en107', 'en108', 'en109', 'en110', 'en111', 'en112', 'en113', 'en114', 'en115', 'en116', 'en117', 'en118', 'en119', 'en120', 'en121', 'en122', 'en123', 'en124', 'en125', 'en126', 'en127', 'eo000', 'es000', 'es100', 'es101', 'es102', 'es103', 'es104', 'es105', 'es106', 'et000', 'eu000', 'fa000', 'ff000', 'fi000', 'fj000', 'fo000', 'fr000', 'fr100', 'fr101', 'fr102', 'fr103', 'fy000', 'ga000', 'gd000', 'gl000', 'gn000', 'gu000', 'ha000', 'hi000', 'hi100', 'ho000', 'hr000', 'ht000', 'hu000', 'hy000', 'ia000', 'id000', 'id100', 'id101', 'ie000', 'ig000', 'ik000', 'is000', 'it000', 'it100', 'it101', 'iu000', 'iw000', 'ja000', 'ja100', 'jv000', 'ka000', 'ki000', 'kk000', 'kl000', 'km000', 'kn000', 'ko000', 'ko100', 'ko101', 'ko102', 'ko103', 'ks000', 'ku000', 'ky000', 'la000', 'lb000', 'lg000', 'ln000', 'lo000', 'lt000', 'lv000', 'mg000', 'mi000', 'mk000', 'ml000', 'mn000', 'mr000', 'ms000', 'my000', 'na000', 'nd000', 'ne000', 'nl000', 'nl100', 'no000', 'nv000', 'oc000', 'om000', 'or000', 'pa000', 'pl000', 'ps000', 'pt000', 'pt100', 'pt101', 'pt102', 'pt103', 'qu000', 'rm000', 'rn000', 'ro000', 'ru000', 'ru001', 'ru100', 'ru101', 'ru102', 'ru103', 'ru104', 'ru105', 'ru106', 'rw000', 'sa000', 'sc000', 'sd000', 'sg000', 'sh000', 'si000', 'sk000', 'sl000', 'sm000', 'sn000', 'so000', 'sq000', 'sr000', 'st000', 'su000', 'sv000', 'sw000', 'ta000', 'te000', 'tg000', 'th000', 'th100', 'ti000', 'tk000', 'tn000', 'to000', 'tr000', 'tr100', 'ts000', 'tt000', 'ug000', 'uk000', 'uk100', 'ur000', 'uz000', 've000', 'vi000', 'vi100', 'vi101', 'vo000', 'wo000', 'xh000', 'yi000', 'yo000', 'zh000', 'zu000']
configs = ['ki000', 'sg000','to000']

# this STREAMING loading will finish quickly, can load only one by one
subset = 'cy000'
ds = load_dataset('espnet/yodas', subset, split="train", streaming=True, trust_remote_code=True)

# #try this for ALL configs in streaming mode (NOT WORKING)
# for i in configs:
#     ds = load_dataset('espnet/yodas', i, streaming=True)

#{'id': '9774', 'utt_id': 'YoRjzEnRcqu-00000-00000716-00000819', 'audio': {'path': None, 'array': array([-0.009552  , -0.01086426, -0.012146  , ..., -0.01992798,
#       -0.01885986, -0.01074219]), 'sampling_rate': 16000}, 'text': 'There is a saying'}
print(ds)
print(next(iter(ds)))

IterableDataset({
    features: ['id', 'utt_id', 'audio', 'text'],
    n_shards: 8
})
{'id': '825', 'utt_id': 'YMHjfSZqz3U-00000-00000394-00000814', 'audio': {'path': None, 'array': array([0.03961182, 0.04116821, 0.03549194, ..., 0.01452637, 0.00314331,
       0.02011108]), 'sampling_rate': 16000}, 'text': 'Wyt ti’n ystyried dechrau cwrs ôl-raddedig yn llawn amser neu’n rhan-amser?'}


## **Download and activate model (openai/whisper-large-v3)**

In [56]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4090 Laptop GPU'

In [44]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=False,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## **Run model with dataset**

In [50]:
import sys
import string
from tqdm import tqdm  # For progress bar
import logging
from difflib import SequenceMatcher

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def preprocess_text(text):
    return text.strip().lower().translate(str.maketrans('', '', string.punctuation))

# def evaluate_transcriptions(sample, pipe):
# if len(results_dict) == 0: 
correct_transcriptions = 0
total = 0
results_ls = []
results_dict = []
results_ls_filtered = []

for i in tqdm(ds, desc="Processing Audio Samples"):
    try:
        result = pipe(i['audio']) #, generate_kwargs={"language": "spanish"})
    except Exception as e: print(e)
            # print(f"Problems with row: {i['id']}")
            # sys.exit(1)

    total += 1
    expected_text = preprocess_text(i['text'])
    transcribed_text = preprocess_text(result['text'])

    ratio = SequenceMatcher(None, expected_text, transcribed_text).ratio()

    if transcribed_text == expected_text: # if texts match!
        matched = 1
        correct_transcriptions += 1
        results_ls_filtered.append(i['id'])
    else:
        matched = 0

    accuracy = correct_transcriptions / total
    results_ls.append([i['id'], i['text'], result['text'], matched, ratio])
    results_dict.append({'id': i['id'], 'exptext': i['text'], 'tratext': result['text'], 'acc': matched, 'ratio': ratio})

    logging.info(f"Processed {total} samples. Current accuracy: {accuracy:.2%}")

final_accuracy = correct_transcriptions / total
logging.info(f"Final Accuracy: {final_accuracy:.2%}")

    # return results_ls, results_dict, final_accuracy

# Example usage (making it function wont return results if breaks)
# sample = ds
# results_ls, results_dict, final_accuracy = evaluate_transcriptions(sample, pipe)

# If needed, print the final accuracy
print(f"Final Accuracy: {final_accuracy:.2%}")

Processing Audio Samples: 0it [00:00, ?it/s]

2024-07-05 17:07:49,400 - INFO - Processed 1 samples. Current accuracy: 0.00%
Processing Audio Samples: 1it [00:01,  1.85s/it]2024-07-05 17:07:50,241 - INFO - Processed 2 samples. Current accuracy: 0.00%
Processing Audio Samples: 2it [00:02,  1.26s/it]2024-07-05 17:07:50,768 - INFO - Processed 3 samples. Current accuracy: 0.00%
Processing Audio Samples: 3it [00:03,  1.08it/s]2024-07-05 17:07:51,594 - INFO - Processed 4 samples. Current accuracy: 0.00%
Processing Audio Samples: 4it [00:04,  1.13it/s]2024-07-05 17:07:52,423 - INFO - Processed 5 samples. Current accuracy: 0.00%
Processing Audio Samples: 5it [00:04,  1.16it/s]2024-07-05 17:07:53,175 - INFO - Processed 6 samples. Current accuracy: 0.00%
Processing Audio Samples: 6it [00:05,  1.21it/s]2024-07-05 17:07:53,733 - INFO - Processed 7 samples. Current accuracy: 0.00%
Processing Audio Samples: 7it [00:06,  1.35it/s]2024-07-05 17:07:54,551 - INFO - Processed 8 samples. Current accuracy: 0.00%
Processing Audio Samples: 8it [00:07,  1

Problems with row: 634


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
for i in results_ls:
    print(i)
    print(len(results_ls))
    break
    
# print(results_dict)

# for i in results_dict:
#     for key in i:
#         print(key, 'corresponds to', i[key])
#     # break

In [ ]:
# from datasets import Dataset

# my_dataset = Dataset.from_list(results_dict)
# print(my_dataset)

In [ ]:
# import string

# sample = ds['train']

# correct_transcriptions = 0
# total = 0
# results_ls = []
# results_dict = []

# for i in sample:
#     result = pipe(i['audio']) #, generate_kwargs={"language": "spanish"})
#     total += 1
#     print(total)
#     # print(i['audio'])
#     print('e text --->', i['text']) #expected
#     print('t text --->', result['text']) #transcribed

#     if result['text'].strip().lower().translate(str.maketrans('','', string.punctuation)) == i['text'].strip().lower():
#         correct_transcriptions += 1

#     accuracy = correct_transcriptions / total
#     # print(f"Accuracy: {accuracy:.2%}")
#     results_ls.append([i['id'],  i['text'], result['text'], accuracy])
#     results_dict.append({'id': i['id'],  'exptext': i['text'], 'tratext': result['text'], 'acc': accuracy})
#     # if correct_transcriptions == 1:
#     #     break

# accuracy = correct_transcriptions / total
# print(f"Accuracy: {accuracy:.2%}")

In [ ]:
# # Get ids where audio does match (100% accuracy)
# results_ls_filtered = []
# for re in results_ls:
#     if re[3] == 1:
#         results_ls_filtered.append(re[0])

# print(len(results_ls_filtered))

In [47]:
# Save IDs to csv file to read later
import csv
 
with open(subset + '.csv', 'w', newline ='') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
     
    write.writerow(results_ls_filtered)

**Open file of IDs filtered**

In [ ]:
# OPEN csv file to start reading from here and stream the dataset with these IDs only
import csv

with open(subset + '.csv', newline='') as f:
    reader = csv.reader(f)
    results_ls_filtered = list(reader)[0]

print(len(results_ls_filtered))

In [ ]:
# get only the audios with 100% accuracy from the original dataset
# filtered_entries = []
# for i in results_ls_filtered:
# #     entry = (next(iter(ds.filter(lambda x: x["id"] == i))))
# #     ds.select(i['id'])
#     filtered_entries.append(entry)

# filtered_dataset = Dataset.from_list(filtered_entries).to_iterable_dataset()
# print(type(filtered_dataset))

# --------------------------------------

# GET BY STREAMING AGAING IF NEEDED
# this STREAMING loading will finish quickly, can load only one by one
# subset = 'en000'
# ds = load_dataset('espnet/yodas', subset, split="train", streaming=True, trust_remote_code=True)


#Original dataset in streaming is filter only for the rows that matched
#THIS LINE CAN BE USED AFTER ds = load_dataset('espnet/yodas'...
filtered_dataset = ds.filter(lambda x: x["id"] in results_ls_filtered)
print(filtered_dataset)
print(next(iter(filtered_dataset)))

In [ ]:
# filtered_dataset = ds.map(lambda x: x['id'] in results_ls_filtered) #x["id"] in results_ls_filtered)
# print(filtered_dataset)
# print(next(iter(filtered_dataset)))

for i in results_ls_filtered:
    print(i)
    break

In [51]:
print(pipe)